**This program will parse your data of midi files into data readable by the ai**

Import

In [1]:
!pip install py_midicsv
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
!rm -rf /content/music/

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [2]:
!unzip /content/music.zip -d /content/music/

Streaming output truncated to the last 5000 lines.
  inflating: /content/music/med music/EltonJohn_BitchIsBack.mid  
  inflating: /content/music/med music/Crazy (2)_1.mid  
  inflating: /content/music/med music/jpw70kmo.mid  
  inflating: /content/music/med music/SPACE (3).MID  
  inflating: /content/music/med music/doda.mid  
  inflating: /content/music/med music/Jesu Joy of Man Desiring.mid  
  inflating: /content/music/med music/UComeAgn.mid  
  inflating: /content/music/med music/baby_Elephant_Walk-Henry_Mancin4i-F115.mid  
  inflating: /content/music/med music/misterio.mid  
  inflating: /content/music/med music/Giuliani_Sonate_Op15_1er_mouvement.mid  
  inflating: /content/music/med music/elegiecl.mid  
  inflating: /content/music/med music/l1.mid  
  inflating: /content/music/med music/c06019.mid  
  inflating: /content/music/med music/Misty (15).mid  
  inflating: /content/music/med music/thewaywe.mid  
  inflating: /content/music/med music/GradeBooMeevies.mid  
  inflating: /c

In [3]:
!mkdir /content/musicFixed/

**Convert the midis into all one track**

In [4]:
import os
import mido

def merge_midi_tracks(input_file):
    try:
      mid = mido.MidiFile("/content/music/" + input_file)

      merged = mido.MidiFile(ticks_per_beat=mid.ticks_per_beat)

      # Create a new track to merge the MIDI events
      merged_track = mido.MidiTrack()
      merged.tracks.append(mido.merge_tracks(mid.tracks))

      # Save the merged MIDI file

      merged.save("/content/musicFixed/" + input_file)
      print(input_file)
    except:
      pass

for filename in os.listdir("/content/music/"):
    if filename.endswith('.mid'):
      merge_midi_tracks(filename)

Streaming output truncated to the last 5000 lines.
only_sad.mid
Diamond-HeAintHeavy.mid
The-Power-Of-Love-1 (2).mid
Bwv0561 Fantasie and Fuga.mid
itsyourlove (2).mid
honeybee2.mid
Bang-Bang-(My-Baby-Shot-Me-Down).mid
sw53hsatr.mid
sleep_1.mid
980217.mid
Norwegian Wood.mid
THE PLATTERS.Love letters in the sand.mid
Coma_Black_[Marilyn Manson].mid
more_than_a_feeling_1.mid
iminthemoodforlove2.mid
walzbl44.mid
Barrios_Mangore_Sueno_Floresta.mid
34eimh.mid
CielitoLindo.mid
violaenl.mid
L'anno che verrà.mid
bman_dc_pso_motherearthofdishonesty1.mid
When_I_Come_Around (2).mid
99smasots.mid
Hand_On_Heart.mid
withyouimbornagain.mid
ambulante.mid
you_light_up_my_life1-C80.mid
right_time_of_the_night1-A103.mid
Tornerò.mid
etuded_1.mid
Bobby_Vinton_-_My_Special_Angel.mid
K495 Horn Concerto n4 3mov.mid
OhMyPapa_1.mid
cho_nguoi_tinh_lo.mid
Jungle (6).mid
brickhouse3.mid
orient04.mid
RighteousBrothers_You'veLostThatLovin'Feeling1.mid
dont_make_promises1-F150-gary_puckett_and_the_union_gap.mid
my_sins_

In [5]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/musicFixed/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  outValue = str(note)

  if(instrument<=127):
    outValue += "%"

  outValue += str(start)

  if(velocity > 108):
    outValue += "$"
  elif velocity > 74:
    outValue += "#"
  elif velocity > 40:
    outValue += "@"
  else:
    outValue += "!"

  outValue += str(length)

  return outValue

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
for midi_file in midi_files:
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    csv_parsed = []
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

for i in range(0, len(csv_string), 1800):
  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

Streaming output truncated to the last 5000 lines.
Event length mismatch for PortEvent at position 73
Event length mismatch for PortEvent at position 640
Event length mismatch for PortEvent at position 278
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 242
Event length mismatch for PortEvent at position 149
Event length mismatch for PortEvent at position 96
Event length mismatch for PortEvent at position 57
Event length mismatch for PortEvent at position 98
Event length mismatch for PortEvent at position 36
Event length mismatch for PortEvent at position 57
Event length mismatch for PortEvent at position 48
Event length mismatch for PortEvent at position 270
Event length mismatch for PortEvent at position 48
Event length mismatch for PortEvent at position 98
Event length mismatch for PortEvent at position 73
Event length mismatch for PortEvent at position 42
Event length mismatch for PortEvent at position 68
Event length mismatch for 